In [2]:
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
dataset = Planetoid("./", "Cora", transform=T.NormalizeFeatures())
data = dataset[0]

x = data.x
edge_index = data.edge_index
edge_weight = data.edge_weight

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class PyG_GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(PyG_GCNConv, self).__init__(aggr='add') 
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        # Step 4-6: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):    
        # Normalize node features.
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):        
        # Step 5: Return new node embeddings.
        return aggr_out

class PyG_GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(PyG_GCN, self).__init__()
        self.conv1 = PyG_GCNConv(in_channels, hidden_channels)
        self.conv2 = PyG_GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [11]:
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

# Build your training pipeline
hidden_dim = 16
lr = 0.001
epochs = 100
model = PyG_GCN(dataset.num_features, hidden_dim, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])  # Use negative log likelihood loss
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

best_val_acc = 0
for epoch in range(1, epochs + 1):
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    logging.info("Epoch: %d, Loss: %.4f, Train Acc: %.4f, Val Acc: %.4f, Test Acc: %.4f", epoch, loss, train_acc, val_acc, test_acc)


INFO:root:Epoch: 1, Loss: 1.9619, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 2, Loss: 1.9612, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 3, Loss: 1.9605, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 4, Loss: 1.9597, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 5, Loss: 1.9590, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 6, Loss: 1.9583, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 7, Loss: 1.9576, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 8, Loss: 1.9569, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 9, Loss: 1.9561, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 10, Loss: 1.9554, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 11, Loss: 1.9547, Train Acc: 0.1429, Val Acc: 0.3160, Test Acc: 0.3190
INFO:root:Epoch: 12, Loss: 1.9539, Train 

INFO:root:Epoch: 95, Loss: 1.8534, Train Acc: 0.7214, Val Acc: 0.4740, Test Acc: 0.5070
INFO:root:Epoch: 96, Loss: 1.8518, Train Acc: 0.7214, Val Acc: 0.4880, Test Acc: 0.5220
INFO:root:Epoch: 97, Loss: 1.8501, Train Acc: 0.7214, Val Acc: 0.4940, Test Acc: 0.5260
INFO:root:Epoch: 98, Loss: 1.8483, Train Acc: 0.7286, Val Acc: 0.4980, Test Acc: 0.5350
INFO:root:Epoch: 99, Loss: 1.8466, Train Acc: 0.7643, Val Acc: 0.4960, Test Acc: 0.5350
INFO:root:Epoch: 100, Loss: 1.8449, Train Acc: 0.7571, Val Acc: 0.5000, Test Acc: 0.5490


In [3]:
import argparse

import dgl
import dgl.nn as dglnn

import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import AddSelfLoop
from dgl.data import CoraGraphDataset

transform = (
        AddSelfLoop()
    )
data = CoraGraphDataset(transform=transform)
g = data[0]
features = g.ndata["feat"]
labels = g.ndata["label"]
masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]

class DGL_GCNConv(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(DGL_GCNConv, self).__init__()
        self.conv = dglnn.GraphConv(in_feats, out_feats)
    
    def forward(self, g, features):
        return self.conv(g, features)

class DGL_GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(DGL_GCN, self).__init__()
        self.conv1 = DGL_GCNConv(in_feats, hidden_size)
        self.conv2 = DGL_GCNConv(hidden_size, num_classes)
    
    def forward(self, g, features):
        x = F.relu(self.conv1(g, features))
        x = self.conv2(g, x)
        return x

def train(g, features, labels, masks, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    train_mask, val_mask, test_mask = masks
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(200):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            val_acc = evaluate(g, features, labels, val_mask, model)
            print(f"Epoch {epoch}, Loss: {loss.item()}, Validation Accuracy: {val_acc:.4f}")

def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

model = DGL_GCN(features.shape[1], 16, data.num_classes)
print("Training...")
train(g, features, labels, masks, model)

# test the model
print("Testing...")
acc = evaluate(g, features, labels, masks[2], model)
print("Test accuracy {:.4f}".format(acc))


C:\Users\31293\.dgl\cora_v2.zip: 100%|███████████████████████████████████████████████| 132k/132k [00:00<00:00, 796kB/s]


Extracting file to C:\Users\31293\.dgl\cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Training...
Epoch 0, Loss: 1.945562720298767, Validation Accuracy: 0.3300
Epoch 20, Loss: 1.5899418592453003, Validation Accuracy: 0.6940
Epoch 40, Loss: 0.9424986839294434, Validation Accuracy: 0.7400
Epoch 60, Loss: 0.41126778721809387, Validation Accuracy: 0.7520
Epoch 80, Loss: 0.17714110016822815, Validation Accuracy: 0.7640
Epoch 100, Loss: 0.0898863673210144, Validation Accuracy: 0.7660
Epoch 120, Loss: 0.05379465967416763, Validation Accuracy: 0.7720
Epoch 140, Loss: 0.03615487366914749, Validation Accuracy: 0.7720
Epoch 160, Loss: 0.026218481361865997, Validation Accuracy: 0.7720
Epoch 180, Loss: 0.0200169887393713, Validation Accuracy: 0.7740
Testing...
Test accuracy 0.7790
